In [1]:
import pandas as pd
import numpy as np
import keras
import scipy
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn import linear_model
from lineartree import RandomForestRegressor
from sklearn import metrics
from scipy.signal import savgol_filter
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [2]:
from numpy.random import seed
seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(2)

In [3]:
def rae(actual, predicted):
    actual = np.asarray(actual).reshape(-1,1)
    predicted = np.asarray(predicted).reshape(-1,1)
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

In [4]:
def peak_eV_nm(peak_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    return 1239.84193/peak_eV

In [5]:
def FWHM_eV_nm(peak_eV,fwhm_eV):
    peak_eV = np.asarray(peak_eV).reshape(-1,1)
    fwhm_eV = np.asarray(fwhm_eV).reshape(-1,1)
    return  2*(1239.84193/(peak_eV-fwhm_eV/2) - 1239.84193/(peak_eV))

In [6]:
def predict_dist(X, model, num_samples):
    preds = [model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

In [7]:
def predict_point(X, model, num_samples):
    pred_dist = predict_dist(X, model, num_samples)
    return pred_dist.mean(axis=1)

In [8]:
#Define models

initializer = keras.initializers.HeNormal(seed=1)
opt = keras.optimizers.Adam(learning_rate=0.001)

model_peak = keras.models.Sequential()
model_peak.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(50,  activation='relu'))
model_peak.add(keras.layers.Dropout(0.2))
model_peak.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_peak.compile(loss='mean_squared_error', optimizer= opt)

model_fwhm = keras.models.Sequential()
model_fwhm.add(Dense(5, input_shape=(5,), kernel_initializer=initializer, activation='relu'))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(50,  activation='relu'))
model_fwhm.add(keras.layers.Dropout(0.2))
model_fwhm.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_fwhm.compile(loss='mean_squared_error', optimizer= opt)

model_cascade = keras.models.Sequential()
model_cascade.add(Dense(6, input_shape=(6,), kernel_initializer=initializer, activation='relu'))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(50,  activation='relu'))
model_cascade.add(keras.layers.Dropout(0.2))
model_cascade.add(Dense(1, kernel_initializer=initializer))
# Compile model
model_cascade.compile(loss='mean_squared_error', optimizer= opt)


In [9]:
#Xval on Train/Val set, 10-fold cross validation (Group splits), UV Peak, FWHM (eV), Normalized
# load dataset into Pandas DataFrame (cleaned UV data)
df = pd.read_csv('CdSe UV Train Val.csv')

#normalize input and output data
min_params = pd.Series([0, 0, 3, 200, 0, 400, 15, 1239.84193/620, 0.05], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
max_params = pd.Series([50, 50, 150, 310, 600, 620, 80, 1239.84193/400, 0.4], index =['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV'])
df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] = (df[['Cd_mM', 'Se_mM', 'OA_Cd_ratio', 'T_reactor', 't_s','Peak_nm','FWHM_nm','Peak_eV','FWHM_eV']] - min_params) / (max_params- min_params)

X = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s']]
X2 = df[['OA_Cd_ratio','T_reactor','Cd_mM','Se_mM','t_s','Peak_eV']]
y = df['Peak_eV']
z = df['FWHM_eV']
group = df['Expt. No']

#Split groups for cross validation by Experiment number
gkf = GroupKFold(n_splits=10)
matrix = np.zeros([10,16])
matrix_fwhm = np.zeros([10,16])
matrix_cascade = np.zeros([10,16])

j = 0
#Loop through the 10 splits
for train_index, test_index in gkf.split(X, groups=group):
    #Separate out training and test sets
    X_train = X.iloc[train_index, :]
    X2_train = X2.iloc[train_index, :]
    y_train = y[train_index]
    z_train = z[train_index]
    X_test = X.iloc[test_index, :]
    X2_test = X2.iloc[test_index, :]
    y_test = y[test_index]
    z_test = z[test_index]
    
    #Load weights
    model_peak.load_weights('initialize_weights3.h5')
    model_fwhm.load_weights('initialize_weights3.h5')
    model_cascade.load_weights('initialize_weights4.h5')

    #Train models
    #prediction for Peak
    model_peak.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred = model_peak.predict(X_test)
    #prediction for FWHM (direct)
    model_fwhm.fit(X_train, z_train, epochs=300, validation_data=(X_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    y_pred_fwhm = model_fwhm.predict(X_test)
    #prediction for FWHM (cascade)
    model_cascade.fit(X2_train, z_train, epochs=300, validation_data=(X2_test, z_test),verbose = 1, use_multiprocessing=False, workers = 100)
    X_test.insert(5, "Peak_eV", model_peak.predict(X_test), True)
    y_pred_cascade = model_cascade.predict(X_test)
    
    #prediction for Peak with dropout
    X_test = X.iloc[test_index, :]
    X_test_t = K.constant(X_test)
    y_pred_d =  predict_point(X_test_t, model_peak, 1000)
    #prediction for FWHM (direct)
    y_pred_fwhm_d = predict_point(X_test_t, model_fwhm, 1000)
    #prediction for FWHM (cascade)
    X_test.insert(5, "Peak_eV", y_pred_d, True)
    X_test_t2 = K.constant(X_test)
    y_pred_cascade_d = predict_point(X_test_t2, model_cascade, 1000)
    
    #errors in eV (normalized)
    #UN-normalize eV predictions by scaling with ratio

    ratio = (1239.84193/400 - 1239.84193/620)
    ratio2 = 0.35
    matrix[j,0] = metrics.mean_absolute_error(y_test, y_pred)*ratio
    matrix[j,1] = metrics.r2_score(y_test, y_pred)
    matrix[j,2] = rae(y_test, y_pred)
    matrix[j,3] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    matrix[j,4] = metrics.mean_absolute_error(y_test, y_pred_d)*ratio 
    matrix[j,5] = metrics.r2_score(y_test, y_pred_d)
    matrix[j,6] = rae(y_test, y_pred_d)
    matrix[j,7] = ratio*np.sqrt(metrics.mean_squared_error(y_test, y_pred_d))

    matrix_fwhm[j,0] = metrics.mean_absolute_error(z_test, y_pred_fwhm)*ratio2
    matrix_fwhm[j,1] = metrics.r2_score(z_test, y_pred_fwhm)
    matrix_fwhm[j,2] = rae(z_test, y_pred_fwhm)
    matrix_fwhm[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm))
    matrix_fwhm[j,4] = metrics.mean_absolute_error(z_test, y_pred_fwhm_d)*ratio2
    matrix_fwhm[j,5] = metrics.r2_score(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,6] = rae(z_test, y_pred_fwhm_d)
    matrix_fwhm[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_fwhm_d))
    
    matrix_cascade[j,0] = metrics.mean_absolute_error(z_test, y_pred_cascade)*ratio2
    matrix_cascade[j,1] = metrics.r2_score(z_test, y_pred_cascade)
    matrix_cascade[j,2] = rae(z_test, y_pred_cascade)
    matrix_cascade[j,3] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade))
    matrix_cascade[j,4] = metrics.mean_absolute_error(z_test, y_pred_cascade_d)*ratio2
    matrix_cascade[j,5] = metrics.r2_score(z_test, y_pred_cascade_d)
    matrix_cascade[j,6] = rae(z_test, y_pred_cascade_d)
    matrix_cascade[j,7] = ratio2*np.sqrt(metrics.mean_squared_error(z_test, y_pred_cascade_d))
    
    

    #errors in nm
    #UNnormalize the data first
    y_test = y_test*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred = y_pred*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    y_pred_d = y_pred_d*(1239.84193/400 - 1239.84193/620)+1239.84193/620
    z_test = z_test*(0.35)+0.05
    y_pred_fwhm = y_pred_fwhm*(0.35)+0.05
    y_pred_fwhm_d = y_pred_fwhm_d*(0.35)+0.05
    y_pred_cascade = y_pred_cascade*(0.35)+0.05
    y_pred_cascade_d = y_pred_cascade_d*(0.35)+0.05
    
    #Calculate data in nm and eV
    matrix[j,8] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,9] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,10] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred))
    matrix[j,11] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred)))
    matrix[j,12] = metrics.mean_absolute_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,13] = metrics.r2_score(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,14] = rae(peak_eV_nm(y_test), peak_eV_nm(y_pred_d))
    matrix[j,15] = np.sqrt(metrics.mean_squared_error(peak_eV_nm(y_test), peak_eV_nm(y_pred_d)))

    matrix_fwhm[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm))
    matrix_fwhm[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm)))
    matrix_fwhm[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))
    matrix_fwhm[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d)) 
    matrix_fwhm[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_fwhm_d))) 

    matrix_cascade[j,8] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,9] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,10] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade))
    matrix_cascade[j,11] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade)))
    matrix_cascade[j,12] = metrics.mean_absolute_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,13] = metrics.r2_score(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,14] = rae(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d))
    matrix_cascade[j,15] = np.sqrt(metrics.mean_squared_error(FWHM_eV_nm(y_test,z_test), FWHM_eV_nm(y_pred,y_pred_cascade_d)))

    j+=1

matrix_2 = np.zeros([1,32])
matrix_2[0,:16] = matrix.mean(0)[:16]
matrix_2[0,16:32] = np.std(matrix[:,:16], axis=0)

matrix_3 = np.zeros([1,32])
matrix_3[0,:16] = matrix_fwhm.mean(0)[:16]
matrix_3[0,16:32]= np.std(matrix_fwhm[:,:16], axis=0)

matrix_4 = np.zeros([1,32])
matrix_4[0,:16] = matrix_cascade.mean(0)[:16]
matrix_4[0,16:32] = np.std(matrix_cascade[:,:16], axis=0)

np.set_printoptions(precision=5, threshold=5, edgeitems=4, suppress=True)


Epoch 1/300
71/71 [==============================] - 1s 4ms/step - loss: 0.0633 - val_loss: 0.0270
Epoch 2/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0220 - val_loss: 0.0186
Epoch 3/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0148 - val_loss: 0.0165
Epoch 4/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0116 - val_loss: 0.0143
Epoch 5/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0099 - val_loss: 0.0124
Epoch 6/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0086 - val_loss: 0.0112
Epoch 7/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0081 - val_loss: 0.0098
Epoch 8/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0073 - val_loss: 0.0093
Epoch 9/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0070 - val_loss: 0.0079
Epoch 10/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0064 - val_loss: 0.0070
Epoch 11/

71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 84/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 85/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 86/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 87/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0015
Epoch 88/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0018
Epoch 89/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 90/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 91/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 92/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0017 - val_loss: 0.0015
Epoch 93/300


71/71 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 165/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 166/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 167/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 168/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 169/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 170/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 171/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0017
Epoch 172/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 173/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0013
Epoc

71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 246/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 247/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0019
Epoch 248/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 249/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 250/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 251/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0013
Epoch 252/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 253/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0014 - val_loss: 0.0020
Epoch 254/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0013 - val_loss: 0.0014
Epoc

71/71 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 27/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0027
Epoch 28/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0030
Epoch 29/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0026
Epoch 30/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0029
Epoch 31/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0030
Epoch 32/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0023 - val_loss: 0.0027
Epoch 33/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 34/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 35/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0022 - val_loss: 0.0032
Epoch 36/300


...

71/71 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 239/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 240/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0025
Epoch 241/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 242/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0023
Epoch 243/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0025
Epoch 244/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 245/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 246/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0022
Epoch 247/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0022
Epoc

Epoch 19/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0029 - val_loss: 0.0019
Epoch 20/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0019
Epoch 21/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 22/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0028 - val_loss: 0.0018
Epoch 23/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0027 - val_loss: 0.0018
Epoch 24/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0027 - val_loss: 0.0018
Epoch 25/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 26/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0017
Epoch 27/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 28/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0017


Epoch 101/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 102/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0011
Epoch 103/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 104/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0013
Epoch 105/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 106/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0013
Epoch 107/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 108/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 109/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0015 - val_loss: 0.0013
Epoch 110/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0015 - val_los

71/71 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 183/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 184/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 185/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 186/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 187/300
71/71 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 188/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 189/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 190/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 191/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0011
Epoc

Epoch 263/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0012
Epoch 264/300
71/71 [==============================] - 0s 1ms/step - loss: 9.5693e-04 - val_loss: 0.0012
Epoch 265/300
71/71 [==============================] - 0s 1ms/step - loss: 9.8896e-04 - val_loss: 0.0012
Epoch 266/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 9.8858e-04
Epoch 267/300
71/71 [==============================] - 0s 1ms/step - loss: 9.6151e-04 - val_loss: 0.0011
Epoch 268/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0012
Epoch 269/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 9.9902e-04
Epoch 270/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0011
Epoch 271/300
71/71 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 272/300
71/71 [==============================] - 0s 1ms/step - lo

In [10]:
print('\033[1m' + 'Model without Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,0]), "{:.4f}".format(matrix_2[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,1]), "{:.4f}".format(matrix_2[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,2]), "{:.4f}".format(matrix_2[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,3]), "{:.4f}".format(matrix_2[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,8]), "{:.4f}".format(matrix_2[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,9]), "{:.4f}".format(matrix_2[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,10]), "{:.4f}".format(matrix_2[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,11]), "{:.4f}".format(matrix_2[0,27]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,0]), "{:.4f}".format(matrix_3[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,1]), "{:.4f}".format(matrix_3[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,2]), "{:.4f}".format(matrix_3[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,3]), "{:.4f}".format(matrix_3[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,8]), "{:.4f}".format(matrix_3[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,9]), "{:.4f}".format(matrix_3[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,10]), "{:.4f}".format(matrix_3[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,11]), "{:.4f}".format(matrix_3[0,27]))

print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,0]), "{:.4f}".format(matrix_4[0,16]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,1]), "{:.4f}".format(matrix_4[0,17]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,2]), "{:.4f}".format(matrix_4[0,18]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,3]), "{:.4f}".format(matrix_4[0,19]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,8]), "{:.4f}".format(matrix_4[0,24]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,9]), "{:.4f}".format(matrix_4[0,25]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,10]), "{:.4f}".format(matrix_4[0,26]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,11]), "{:.4f}".format(matrix_4[0,27]))

Model without Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0315 0.0116
R2 (eV):	 0.9309 0.0551
RAE (eV):	 0.2097 0.0859
RMSE (eV):	 0.0441 0.0141
MAE (nm):	 6.6334 2.4522
R2 (nm):	 0.9243 0.0638
RAE (nm):	 0.2120 0.0869
RMSE (nm):	 9.4546 3.1873
Absorbance FWHM (direct)
MAE (eV): 	 0.0115 0.0023
R2 (eV):	 0.4243 0.2465
RAE (eV):	 0.5879 0.0892
RMSE (eV):	 0.0187 0.0050
MAE (nm):	 2.8518 0.6111
R2 (nm):	 0.4554 0.2733
RAE (nm):	 0.5544 0.1136
RMSE (nm):	 4.5746 1.2315
Absorbance FWHM (cascade)
MAE (eV): 	 0.0094 0.0014
R2 (eV):	 0.6173 0.1189
RAE (eV):	 0.4816 0.0589
RMSE (eV):	 0.0154 0.0033
MAE (nm):	 2.2261 0.3018
R2 (nm):	 0.6776 0.0989
RAE (nm):	 0.4344 0.0640
RMSE (nm):	 3.5627 0.5875


In [11]:
print('\033[1m' + 'Model with Dropout at Inference' + '\033[0m')
print('\033[1m' + 'Absorbance Peak' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_2[0,4]), "{:.4f}".format(matrix_2[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_2[0,5]), "{:.4f}".format(matrix_2[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_2[0,6]), "{:.4f}".format(matrix_2[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_2[0,7]), "{:.4f}".format(matrix_2[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_2[0,12]), "{:.4f}".format(matrix_2[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_2[0,13]), "{:.4f}".format(matrix_2[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_2[0,14]), "{:.4f}".format(matrix_2[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_2[0,15]), "{:.4f}".format(matrix_2[0,31]))

print('\033[1m' + 'Absorbance FWHM (direct)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_3[0,4]), "{:.4f}".format(matrix_3[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_3[0,5]), "{:.4f}".format(matrix_3[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_3[0,6]), "{:.4f}".format(matrix_3[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_3[0,7]), "{:.4f}".format(matrix_3[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_3[0,12]), "{:.4f}".format(matrix_3[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_3[0,13]), "{:.4f}".format(matrix_3[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_3[0,14]), "{:.4f}".format(matrix_3[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_3[0,15]), "{:.4f}".format(matrix_3[0,31]))
print('\033[1m' + 'Absorbance FWHM (cascade)' + '\033[0m')
print("MAE (eV): \t", "{:.4f}".format(matrix_4[0,4]), "{:.4f}".format(matrix_4[0,20]))
print("R2 (eV):\t", "{:.4f}".format(matrix_4[0,5]), "{:.4f}".format(matrix_4[0,21]))
print("RAE (eV):\t", "{:.4f}".format(matrix_4[0,6]), "{:.4f}".format(matrix_4[0,22]))
print("RMSE (eV):\t", "{:.4f}".format(matrix_4[0,7]), "{:.4f}".format(matrix_4[0,23]))

print("MAE (nm):\t","{:.4f}".format(matrix_4[0,12]), "{:.4f}".format(matrix_4[0,28]))
print("R2 (nm):\t", "{:.4f}".format(matrix_4[0,13]), "{:.4f}".format(matrix_4[0,29]))
print("RAE (nm):\t","{:.4f}".format(matrix_4[0,14]), "{:.4f}".format(matrix_4[0,30]))
print("RMSE (nm):\t", "{:.4f}".format(matrix_4[0,15]), "{:.4f}".format(matrix_4[0,31]))

Model with Dropout at Inference
Absorbance Peak
MAE (eV): 	 0.0312 0.0110
R2 (eV):	 0.9306 0.0559
RAE (eV):	 0.2077 0.0827
RMSE (eV):	 0.0441 0.0143
MAE (nm):	 6.5573 2.3331
R2 (nm):	 0.9239 0.0655
RAE (nm):	 0.2097 0.0836
RMSE (nm):	 9.4374 3.2608
Absorbance FWHM (direct)
MAE (eV): 	 0.0113 0.0024
R2 (eV):	 0.4270 0.2489
RAE (eV):	 0.5784 0.0951
RMSE (eV):	 0.0186 0.0051
MAE (nm):	 2.8139 0.6217
R2 (nm):	 0.4596 0.2760
RAE (nm):	 0.5471 0.1156
RMSE (nm):	 4.5481 1.2695
Absorbance FWHM (cascade)
MAE (eV): 	 0.0091 0.0014
R2 (eV):	 0.6343 0.1133
RAE (eV):	 0.4686 0.0564
RMSE (eV):	 0.0151 0.0032
MAE (nm):	 2.1877 0.3025
R2 (nm):	 0.6887 0.0946
RAE (nm):	 0.4268 0.0634
RMSE (nm):	 3.5008 0.5649


In [12]:
np.savetxt("NN+MMCN_UV_Val_peak.csv", matrix, delimiter=",")
np.savetxt("NN+MMCN_UV_Val_fwhm.csv", matrix_fwhm, delimiter=",")
np.savetxt("NN+MMCN_UV_Val_cascade.csv", matrix_cascade, delimiter=",")